In [1]:
import pandas as pd
import os

In [3]:
reg_off_df = pd.read_csv('nba_team_stats_data/nba_off.csv')
reg_def_df = pd.read_csv('nba_team_stats_data/nba_def.csv')
reg_adv_df = pd.read_csv('nba_team_stats_data/nba_adv_stats.csv')
post_off_df = pd.read_csv('nba_team_stats_data/nba_post_off.csv')
post_def_df = pd.read_csv('nba_team_stats_data/nba_post_def.csv')
post_adv_df = pd.read_csv('nba_team_stats_data/nba_post_adv_stats.csv')

In [9]:
reg_off_filtered_df = reg_off_df.drop(columns=['Rk'])
reg_off_filtered_df.columns

Index(['Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

In [10]:
reg_def_filtered_df = reg_def_df.drop(columns=['Rk'])
reg_def_filtered_df.columns

Index(['Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

In [8]:
reg_adv_filtered_df = reg_adv_df.drop(columns=['Rk', 'Unnamed: 17', 'Unnamed: 27', 'Arena', 'Attend.'])
reg_adv_filtered_df.columns

Index(['Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%',
       'FT/FGA', 'Unnamed: 22', 'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1',
       'Attend./G'],
      dtype='object')

In [12]:
post_off_filtered_df = post_off_df.drop(columns=['Rk'])
post_off_filtered_df.columns

Index(['Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

In [14]:
post_def_filtered_df = post_def_df.drop(columns=['Rk'])
post_def_filtered_df.columns

Index(['Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

In [16]:
post_adv_filtered_df = post_adv_df.drop(columns=['Rk', 'Unnamed: 15', 'Unnamed: 20'])
post_adv_filtered_df.columns

Index(['Team', 'Age', 'W', 'L', 'W/L%', 'PW', 'PL', 'ORtg', 'DRtg', 'NRtg',
       'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA',
       'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1'],
      dtype='object')

In [38]:
regular_season_df = pd.read_csv('nba_team_stats_data/nba_reg_compiled_stats.csv')
regular_season_df.columns


Index(['Unnamed: 0', 'Team', 'MP', 'Off_FG', 'Off_FGA', 'Off_FG%', 'Off_3P',
       'Off_3PA', 'Off_3P%', 'Off_2P', 'Off_2PA', 'Off_2P%', 'Off_FT',
       'Off_FTA', 'Off_FT%', 'Off_ORB', 'Off_DRB', 'Off_TRB', 'Off_AST',
       'Off_STL', 'Off_BLK', 'Off_TOV', 'Off_PF', 'Off_PTS', 'Def_FG',
       'Def_FGA', 'Def_FG%', 'Def_3P', 'Def_3PA', 'Def_3P%', 'Def_2P',
       'Def_2PA', 'Def_2P%', 'Def_FT', 'Def_FTA', 'Def_FT%', 'Def_ORB',
       'Def_DRB', 'Def_TRB', 'Def_AST', 'Def_STL', 'Def_BLK', 'Def_TOV',
       'Def_PF', 'Def_PTS', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS',
       'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'Off_eFG%',
       'Off_TOV%', 'Off_ORB%', 'Off_FT/FGA', 'Def_eFG%', 'Def_TOV%',
       'Def_DRB%', 'Def_FT/FGA', 'Attend./G'],
      dtype='object')

In [39]:
regular_season_df = regular_season_df.drop(['Unnamed: 0'],axis=1)
regular_season_df.columns = regular_season_df.columns.str.lower()

In [40]:
regular_season_df.columns

Index(['team', 'mp', 'off_fg', 'off_fga', 'off_fg%', 'off_3p', 'off_3pa',
       'off_3p%', 'off_2p', 'off_2pa', 'off_2p%', 'off_ft', 'off_fta',
       'off_ft%', 'off_orb', 'off_drb', 'off_trb', 'off_ast', 'off_stl',
       'off_blk', 'off_tov', 'off_pf', 'off_pts', 'def_fg', 'def_fga',
       'def_fg%', 'def_3p', 'def_3pa', 'def_3p%', 'def_2p', 'def_2pa',
       'def_2p%', 'def_ft', 'def_fta', 'def_ft%', 'def_orb', 'def_drb',
       'def_trb', 'def_ast', 'def_stl', 'def_blk', 'def_tov', 'def_pf',
       'def_pts', 'age', 'w', 'l', 'pw', 'pl', 'mov', 'sos', 'srs', 'ortg',
       'drtg', 'nrtg', 'pace', 'ftr', '3par', 'ts%', 'off_efg%', 'off_tov%',
       'off_orb%', 'off_ft/fga', 'def_efg%', 'def_tov%', 'def_drb%',
       'def_ft/fga', 'attend./g'],
      dtype='object')

In [41]:

regular_season_df = regular_season_df.rename(columns = {'off_fg%':'off_fg_percent','off_3p%':'off_3p_percent','off_2p%':'off_2p_percent','off_ft%':'off_ft_percent',
                                     'def_fg%':'def_fg_percent','def_3p%':'def_3p_percent','def_2p%':'def_2p_percent',
                                    'def_ft%':'def_ft_percent','w/l%':'w_l_percent','ts%':'ts_percent','off_efg%':'off_efg_percent',
                                    'off_tov%':'off_tov_percent','off_orb%':'off_orb_percent','off_ft/fga':'off_ft_fga','def_efg%':'def_efg_percent',
                                    'def_tov%':'def_tov_percent','def_drb%':'def_drb_percent','def_ft/fga':'def_ft_fga', '3par':'three_par',
                                    'attend./g':'attend_g'})
regular_season_df.columns

Index(['team', 'mp', 'off_fg', 'off_fga', 'off_fg_percent', 'off_3p',
       'off_3pa', 'off_3p_percent', 'off_2p', 'off_2pa', 'off_2p_percent',
       'off_ft', 'off_fta', 'off_ft_percent', 'off_orb', 'off_drb', 'off_trb',
       'off_ast', 'off_stl', 'off_blk', 'off_tov', 'off_pf', 'off_pts',
       'def_fg', 'def_fga', 'def_fg_percent', 'def_3p', 'def_3pa',
       'def_3p_percent', 'def_2p', 'def_2pa', 'def_2p_percent', 'def_ft',
       'def_fta', 'def_ft_percent', 'def_orb', 'def_drb', 'def_trb', 'def_ast',
       'def_stl', 'def_blk', 'def_tov', 'def_pf', 'def_pts', 'age', 'w', 'l',
       'pw', 'pl', 'mov', 'sos', 'srs', 'ortg', 'drtg', 'nrtg', 'pace', 'ftr',
       'three_par', 'ts_percent', 'off_efg_percent', 'off_tov_percent',
       'off_orb_percent', 'off_ft_fga', 'def_efg_percent', 'def_tov_percent',
       'def_drb_percent', 'def_ft_fga', 'attend_g'],
      dtype='object')

In [10]:

post_season_df = pd.read_csv('nba_team_stats_data/nba_post_compiled_stats.csv')
post_season_df.columns

Index(['Unnamed: 0', 'Team', 'MP', 'Off_FG', 'Off_FGA', 'Off_FG%', 'Off_3P',
       'Off_3PA', 'Off_3P%', 'Off_2P', 'Off_2PA', 'Off_2P%', 'Off_FT',
       'Off_FTA', 'Off_FT%', 'Off_ORB', 'Off_DRB', 'Off_TRB', 'Off_AST',
       'Off_STL', 'Off_BLK', 'Off_TOV', 'Off_PF', 'Off_PTS', 'Def_FG',
       'Def_FGA', 'Def_FG%', 'Def_3P', 'Def_3PA', 'Def_3P%', 'Def_2P',
       'Def_2PA', 'Def_2P%', 'Def_FT', 'Def_FTA', 'Def_FT%', 'Def_ORB',
       'Def_DRB', 'Def_TRB', 'Def_AST', 'Def_STL', 'Def_BLK', 'Def_TOV',
       'Def_PF', 'Def_PTS', 'Age', 'W', 'L', 'W/L%', 'PW', 'PL', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'Off_eFG%', 'Off_TOV%',
       'Off_ORB%', 'Off_FT/FGA', 'Def_eFG%', 'Def_TOV%', 'Def_DRB%',
       'Def_FT/FGA'],
      dtype='object')

In [11]:
post_season_df = post_season_df.drop(['Unnamed: 0'],axis=1)
post_season_df.columns = post_season_df.columns.str.lower()

In [ ]:
{'off_fg%':'off_fg_percent','off_3p%':'off_3p_percent','off_2p%':'off_2p_percent','off_ft%':'off_ft_percent',
                                     'def_fg%':'def_fg_percent','def_3p%':'def_3p_percent','def_2p%':'def_2p_percent',
                                    'def_ft%':'def_ft_percent','w/l%':'w_l_percent','ts%':'ts_percent','off_efg%':'off_efg_percent',
                                    'off_tov%':'off_tov_percent','off_orb%':'off_orb_percent','off_ft/fga':'off_ft_fga','def_efg%':'def_efg_percent',
                                    'def_tov%':'def_tov_percent','def_drb%':'def_drb_percent','def_ft/fga':'def_ft_fga', '3par':'three_par',
                                    'attend./g':'attend_g'})

In [12]:

post_season_df = post_season_df.rename(columns = {'off_fg%':'off_fg_percent','off_3p%':'off_3p_percent','off_2p%':'off_2p_percent','off_ft%':'off_ft_percent',
                                     'def_fg%':'def_fg_percent','def_3p%':'def_3p_percent','def_2p%':'def_2p_percent',
                                    'def_ft%':'def_ft_percent','w/l%':'w_l_percent','ts%':'ts_percent','off_efg%':'off_efg_percent',
                                    'off_tov%':'off_tov_percent','off_orb%':'off_orb_percent','off_ft/fga':'off_ft_fga','def_efg%':'def_efg_percent',
                                    'def_tov%':'def_tov_percent','def_drb%':'def_drb_percent','def_ft/fga':'def_ft_fga','3par':'three_par',
                                    'attend./g':'attend_g'})
post_season_df.columns

Index(['team', 'mp', 'off_fg', 'off_fga', 'off_fg_percent', 'off_3p',
       'off_3pa', 'off_3p_percent', 'off_2p', 'off_2pa', 'off_2p_percent',
       'off_ft', 'off_fta', 'off_ft_percent', 'off_orb', 'off_drb', 'off_trb',
       'off_ast', 'off_stl', 'off_blk', 'off_tov', 'off_pf', 'off_pts',
       'def_fg', 'def_fga', 'def_fg_percent', 'def_3p', 'def_3pa',
       'def_3p_percent', 'def_2p', 'def_2pa', 'def_2p_percent', 'def_ft',
       'def_fta', 'def_ft_percent', 'def_orb', 'def_drb', 'def_trb', 'def_ast',
       'def_stl', 'def_blk', 'def_tov', 'def_pf', 'def_pts', 'age', 'w', 'l',
       'w_l_percent', 'pw', 'pl', 'ortg', 'drtg', 'nrtg', 'pace', 'ftr',
       'three_par', 'ts_percent', 'off_efg_percent', 'off_tov_percent',
       'off_orb_percent', 'off_ft_fga', 'def_efg_percent', 'def_tov_percent',
       'def_drb_percent', 'def_ft_fga'],
      dtype='object')

In [13]:
teams_df = post_season_df[['team']]
teams_df = teams_df.rename(columns = {'team':'team_name'})
teams_df.head()

,team_name
0,Golden State Warriors
1,Memphis Grizzlies
2,Denver Nuggets
3,New Orleans Pelicans
4,Minnesota Timberwolves


In [8]:
team_names_series = regular_season_df["team"]

team_abbv = ["MIN", "MEM", "MIL", "CHA", "PHX", "ATL", "UTA", "SAS", "BKN", "DEN", "LAL", "BOS", "CHI", "IND", "GSW", "SAC", "MIA", "PHI", "HOU", "TOR", "NOP", "WAS", "LAC", "DAL", "CLE", "NYK", "POR", "DET", "ORL", "OKC"]
team_abbv_series = pd.Series(team_abbv)

frame = { 'team_name': team_names_series, 'abbreviation': team_abbv_series }

name_abbv = pd.DataFrame(frame)

name_abbv

,team_name,abbreviation
0,Minnesota Timberwolves,MIN
1,Memphis Grizzlies,MEM
2,Milwaukee Bucks,MIL
3,Charlotte Hornets,CHA
4,Phoenix Suns,PHX
5,Atlanta Hawks,ATL
6,Utah Jazz,UTA
7,San Antonio Spurs,SAS
8,Brooklyn Nets,BKN
9,Denver Nuggets,DEN


In [6]:
from pprint import pprint
from sqlalchemy import create_engine

In [7]:
rds_connection_string = "postgres:admin@localhost:5432/group_2_project_4"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
engine.table_names()

/var/folders/4s/j3_l93ts54qgnyg57c9s590m0000gn/T/ipykernel_6540/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['teams', 'reg_season_stats', 'post_season_stats']

In [20]:
name_abbv.to_sql(name='teams', con=engine, if_exists='append', index=False)

In [21]:
pd.read_sql_query('select * from teams', con=engine).head()

,team_name,abbreviation
0,Minnesota Timberwolves,MIN
1,Memphis Grizzlies,MEM
2,Milwaukee Bucks,MIL
3,Charlotte Hornets,CHA
4,Phoenix Suns,PHX


In [42]:
regular_season_df.to_sql(name='reg_season_stats', con=engine, if_exists='append', index=False)

In [43]:
pd.read_sql_query('select * from reg_season_stats', con=engine).head()

,team,mp,off_fg,off_fga,off_fg_percent,off_3p,off_3pa,off_3p_percent,off_2p,off_2pa,...,ts_percent,off_efg_percent,off_tov_percent,off_orb_percent,off_ft_fga,def_efg_percent,def_tov_percent,def_drb_percent,def_ft_fga,attend_g
0,Minnesota Timberwolves,241,42,91,0,15,41,0,27,50,...,1,1,12,24,0,1,14,75,0,16028
1,Memphis Grizzlies,241,44,94,0,12,33,0,32,62,...,1,1,11,30,0,1,13,78,0,15775
2,Milwaukee Bucks,241,42,89,0,14,38,0,28,51,...,1,1,12,23,0,1,12,79,0,17453
3,Charlotte Hornets,242,43,91,0,14,38,0,29,53,...,1,1,12,23,0,1,13,75,0,17092
4,Phoenix Suns,241,44,90,0,12,32,0,32,58,...,1,1,12,22,0,1,13,77,0,16175


In [14]:
post_season_df.to_sql(name='post_season_stats', con=engine, if_exists='append', index=False)

In [15]:
pd.read_sql_query('select * from post_season_stats', con=engine).head()

,team,mp,off_fg,off_fga,off_fg_percent,off_3p,off_3pa,off_3p_percent,off_2p,off_2pa,...,ts_percent,off_efg_percent,off_tov_percent,off_orb_percent,off_ft_fga,def_efg_percent,def_tov_percent,def_drb_percent,def_ft_fga,attend_g
0,Golden State Warriors,240,42,86,0,14,36,0,28,50,...,1,1,13,24,0,1,12,78,0,None
1,Memphis Grizzlies,240,40,91,0,13,36,0,27,56,...,1,1,12,26,0,1,16,78,0,None
2,Denver Nuggets,240,39,83,0,11,31,0,28,51,...,1,1,15,28,0,1,12,81,0,None
3,New Orleans Pelicans,240,39,84,0,9,26,0,30,58,...,1,1,13,35,0,1,11,80,0,None
4,Minnesota Timberwolves,240,36,82,0,14,36,0,23,46,...,1,1,16,17,0,1,14,73,0,None
